<a href="https://colab.research.google.com/github/Richardxiao123/TW_stock_analysis/blob/main/%E5%9B%9E%E6%B8%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#載入所有股票代號

In [ ]:
import pandas as pd

stock_code =pd.read_csv("stock_code.csv")
stock_code=stock_code.drop(columns=['stock_code'])
stock_code = stock_code.reset_index().rename(columns={'index': 'stockprice'})

print(stock_code)




# 數據處理


In [ ]:
import yfinance as yf
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

data_test = pd.DataFrame()


for a in stock_code['stockprice']:
    ticker =f"{a}.TW"
    data = yf.download(ticker,start='2024-2-19', end="2024-3-7")

    if data.empty:
        print(f"{ticker}.無資料跳過")
        continue

    # 進行數據標準化
    scaler = MinMaxScaler()
    data_temp = pd.DataFrame(index=data.index)
    data_temp['name'] = f"{a}.TW"

    for x in ['Close', 'High', 'Low', 'Open', 'Volume']:
        data_temp[x] = scaler.fit_transform(data[[x]])  # 線性變換 0~1

    # 10 天為一組轉到同一排
    new_data = {}
    new_data['name'] = [data_temp['name'].iloc[0]]

    for i in ['Close', 'High', 'Low', 'Open']:
        for y in range(10):
            new_data[f"{i}{y+1}"] = data_temp[i].iloc[y]

    new_data = pd.DataFrame(new_data)
    data_test = pd.concat([data_test, new_data], ignore_index=True)
    #print(new_data)

print(data_test)


In [ ]:
from google.colab import files
data_test.to_csv('data_run.csv')
files.download('data_run.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# 開始預測

In [ ]:
import pickle
import pandas as pd

with open("rf_model.pkl", "rb") as f:
    rf_model = pickle.load(f)

#讀檔
data_run =pd.read_csv("data_run.csv")

X_test=data_run.drop(columns=['name']).drop(columns=['Unnamed: 0'])
name=data_run['name']
print(X_test)

y_pred = rf_model.predict(X_test)

print(y_pred)

count=0
buy_list=[]
for n in y_pred:
    if n == 5:
        print(name[count])
        buy_list.append(name[count])
        count+=1
    else:
          count+=1

print(buy_list)

# 回測數據

In [ ]:
tickers=['1615.TW','2330.TW']
for ticker in tickers:
  data = yf.download(ticker,start='2024-3-6', end="2024-3-7")

  print(data)


In [ ]:
import pandas as pd
import numpy as np


def augment_data(data: pd.DataFrame, target_size: int, noise_level: float = 0.02) -> pd.DataFrame:

    num_new_samples = target_size - len(data)
    np.random.seed(0)

    # 隨機從原始資料中選擇資料來進行增強
    selected_indices = np.random.choice(data.index, num_new_samples, replace=True)
    new_samples = data.iloc[selected_indices].copy()

    # 對數據進行增強
    for col in data.columns[3:]:  # 忽略 'Unnamed: 0', 'unnamed', 'score'
        original_values = new_samples[col].values
        noise = np.random.normal(0, noise_level, original_values.shape)

        # 隨機選擇 Scaling 或 Shifting
        random_choice = np.random.rand(len(original_values))
        adjusted_values = np.where(
            random_choice < 0.5,
            original_values * (1 + noise),  # 微調 (Scaling)
            original_values + noise  # 加入噪音 (Shifting)
        )

        new_samples[col] = adjusted_values

    # 將原始資料與增強資料合併
    augmented_data = pd.concat([data, new_samples], ignore_index=True)
    return augmented_data